# Liste Simplement Chainée

## Principe

Les listes chainées se libèrent de la contrainte de stocker les éléments dans des emplacements contigus en mémoire.

#### Bénéfice:

Aucune déplacement d'élément n'est nécessaire pour 

* l'insertion 
* la suppression 

#### Coûts:

* stocker explicitement l'emplacement de l'élément suivant
* avancer de $k$ positions = passer $k$ fois à l'élément suivant
* utilisation moins efficace de la mémoire cache

Les éléments de la liste sont stockés indivuellement dans les maillons de la chaine. Chaque maillon stocke

* l'élément
* un pointeur vers le maillon suivant

In [1]:
class Maillon:
    def __init__(self,valeur):
        self.donnee = valeur       
        self.suivant = None 

Cette structure suffit à constuire une liste manuellement 

In [2]:
tete = Maillon(12) 
tete.suivant = Maillon(99) 
tete.suivant.suivant = Maillon(37) 

Nous utilisons une fonction annexe pour afficher cette liste

In [3]:
import include.liste_simple as h; 
Maillon.__str__ = h.afficher_maillon
print(tete) 

12 → 99 → 37 → ⌀


## Parcours

Pour traverser tous les éléments d'une liste, il faut 

* traiter le maillon courant
* passer au maillon suivant
* jusqu'à ce que l'on atteigne un maillon nul

On peut l'écrire récursivement

In [4]:
def traverser(M):
    if M != None:
        print(M.donnee, end = " → ")
        traverser(M.suivant)
    else:
        print("⌀")

In [5]:
traverser(tete)

12 → 99 → 37 → ⌀


on peut l'écrire itérativement 

In [6]:
def traverser(M):
    while M != None:
        print(M.donnee, end = " → ")
        M = M.suivant
    print("⌀")

In [7]:
traverser(tete)

12 → 99 → 37 → ⌀


L'approche itérative est plus appropriée. Une liste peut contenir plus d'éléments que la profondeur de récursion maximale. Les approches récursives sont donc à proscrire. 

## Opérations en tête

Insérer et supprimer en tête sont des opérations simples.

Pour insérer, on doit

* créer un nouveau maillon qui deviendra la tête
* faire de l'ancienne tête le maillon suivant la nouvelle tête

In [8]:
def inserer_en_tete(ancienne_tete, valeur):
    nouvelle_tete = Maillon(valeur)
    nouvelle_tete.suivant = ancienne_tete
    return nouvelle_tete

In [9]:
print(tete)
tete = inserer_en_tete(tete,25)
print(tete)

12 → 99 → 37 → ⌀
25 → 12 → 99 → 37 → ⌀


Supprimer en tête est encore plus simple. Il suffit de remplacer la tête par l'élément suivant

In [10]:
def supprimer_en_tete(ancienne_tete):
    assert(ancienne_tete)
    nouvelle_tete = ancienne_tete.suivant
    return nouvelle_tete

In [11]:
print(tete)
tete = supprimer_en_tete(tete)
print(tete)

25 → 12 → 99 → 37 → ⌀
12 → 99 → 37 → ⌀


Attention, dans un langage où il faut détruire explicitement les objets allouées dynamiquement - comme le C++ - il ne faut pas oublier de détruire le maillon supprimé

## Opérations en position quelconque

Insérer ou supprimer après un maillon connu est également efficace. 

Pour insérer un élément derrière le maillon `M`, il faut

* créer un nouveau maillon
* insérer ce maillon entre `M` et `M.suivant`

In [12]:
def inserer_apres(M, valeur):
    assert(M)
    nouveau = Maillon(valeur)
    nouveau.suivant = M.suivant
    M.suivant = nouveau

In [13]:
print(tete)
inserer_apres(tete.suivant,42)
print(tete)

12 → 99 → 37 → ⌀
12 → 99 → 42 → 37 → ⌀


Pour supprimer l'élément suivant le maillon M, il suivit de faire pointer `M.suivant` vers l'élément suivant celui à supprimer

In [14]:
def supprimer_apres(M):
    assert(M != None and M.suivant != None)
    M.suivant = M.suivant.suivant

In [15]:
print(tete)
supprimer_apres(tete)
print(tete)

12 → 99 → 42 → 37 → ⌀
12 → 42 → 37 → ⌀


Cette opération nous permet de faire des traitement complexes comme d'ôter les éléments se répétants en positions successives (comme `std::unique` en C++)

In [20]:
zeros_un_deux = None
for i in range(5):
    for j in range((i+3)//2):
        zeros_un_deux = inserer_en_tete(zeros_un_deux,i%3)

In [21]:
print(zeros_un_deux)

1 → 1 → 1 → 0 → 0 → 0 → 2 → 2 → 1 → 1 → 0 → ⌀


In [22]:
def oter_repetitions(M):
    while M != None and M.suivant != None:
        if M.donnee == M.suivant.donnee:
            supprimer_apres(M)
        else:
            M = M.suivant

In [23]:
oter_repetitions(zeros_un_deux)
print(zeros_un_deux)

1 → 0 → 2 → 1 → 0 → ⌀


## Classe ListeSimple

S'il est possible de se contenter de la classe (ou structure) `Maillon`, il peut être utile de définir une classe `ListeSimple` qui 

* se souvient de **quel maillon est la tête**. On évite ainsi de devoir écrire `tete = inserer_en_tete(tete,25)`, qui est source d'erreurs si l'on oublie d'affecter le résultat à `tete`



* stocke éventuellement d'autres information comme la **taille de la liste**. Sinon, calculer la taille d'une liste de $n$ éléments est un opération de complexité $\Theta(n)$


In [61]:
class ListeSimple:
    
    def __init__(self):
        self.tete = None
        self.taille = 0
        
    def __str__(self): 
        return h.afficher_maillon(self.tete)

    def inserer_en_tete(self,valeur):
        self.tete = inserer_en_tete(self.tete,valeur)
        self.taille += 1

    def supprimer_en_tete(self):
        assert(self.taille > 0)
        self.tete = supprimer_en_tete(self.tete)
        self.taille -= 1

In [35]:
L = ListeSimple()
for i in range(10):
    L.inserer_en_tete(i%4)
print(L)

1 → 0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀


In [59]:
L.supprimer_en_tete()
print(L)

0 → 3 → 2 → 1 → 0 → 3 → 2 → 1 → 0 → ⌀


In [60]:
print(L.taille)

9


<table style="width: 100%; border: 0px">
<tr style="background-color:white; border:0px">
<td style="width: 120px; border: 0px">
    <img src="https://heig-vd.ch/ResourcePackages/WhiteFox/assets/images/logo-heig-vd.svg" height=200px align=left >
    </td>
    <td style="vertical-align: middle; border: 0px" height=200px>
    <p style="text-align: left">
        <a href="https://ocuisenaire.github.io/ASD1-notebooks/">ASD1 Notebooks on GitHub.io</a>
 </p>        
<p style="text-align: left">
© Olivier Cuisenaire, 2018 </p>
</td>
</tr>
</table>